# Historical Simple Algorithms

In [19]:
import numpy as np
import matplotlib.pyplot as plt

## Data Generation


## Rosenblatt's Perceptron

This learning algorithm involves mimicking simplfied neurons where it fires (value of $1$) when a threshold of values are met, hence:
$$
\sigma(z) =
\begin{cases} 
1 & \text{if } z \ge 0 \\ 
0 & \text{otherwise}
\end{cases}
$$
where
$$z = \vec{w}^T\vec{x} + b$$
The **bias unit** $b$ is $ -\theta$


### Perceptron learning rule
$$\vec{w} = \text{small random numbers}, \ b = 0 \tag{1}$$
For each training example, $x^{(i)}$, compute $\hat{y}^{(i)}$:

$$\hat{y}^{(i)} = \sigma(z^{(i)}) \tag{2}$$
then update weights and bias:
$$w_j \coloneqq w_j + \eta(y^{(i)} - \hat{y}^{(i)})x_j^{(i)}\tag{3}$$
$$b \coloneqq b + \eta(y^{(i)} - \hat{y}^{(i)})$$

Repeat steps $(2)$ & $(3)$ until max epochs and/or max misclassifications is met.



### Implementation of `Perceptron()` Class

In [ ]:
class Perceptron():
    
    def __init__(self, eta, n_epochs):
        self._w = None
        self._b = None
        self._eta = eta
        self._n_epochs = n_epochs
        
    def _net_input(self, X):
        return X @ self._w + self._b
    
    def predict(self, X):
        return np.where(self._net_input(X) >= 0, 1, 0)
    
    def fit(self, X, y):
        m = X.shape[0]
        n = X.shape[1]
        
        self._w = np.random.rand(n)
        self._b = 0
    
        for i in range(self._n_epochs):
            # predict & calc error
            y_pred = self.predict(X)
            error = (y - y_pred).rehshape(-1, 1)
            # update
            self._w = self._w + self._eta * error * X
            self._b = self._b = self._eta * error

In [18]:
a = np.array([0.5, 2])
b = np.array([[2, 2], [3, 3]])
print(a)
print(b)

a.reshape(-1, 1) * b

[0.5 2. ]
[[2 2]
 [3 3]]


array([[1., 1.],
       [6., 6.]])